Boosting Tree

In [1]:
import pandas as pd
newdata = pd.read_csv("newfile.csv")
newdata = newdata.dropna()
newdata.shape


(456759, 22)

In [2]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in ['sex', 'Last_Update','province', 'country','Country_Region','additional_information','source','outcome','Combined_Key','Combined_Key','date_confirmation','Combined_Key','Province_State']:
    le = le.fit(newdata[col])
    newdata[col] = le.transform(newdata[col])

In [3]:
newdata.head(10)

,age,sex,province,country,latitude,longitude,date_confirmation,additional_information,source,outcome,...,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,43.0,2,104,49,23.02776,72.600270,74,7017,866,1,...,0,22.694884,71.590923,120336.0,3286.0,100974.0,16076.0,143,188.400627,2.730687
2,94.0,0,149,84,-12.04318,-77.028240,74,7017,8120,2,...,0,-11.766533,-76.604498,349167.0,14009.0,0.0,335158.0,201,3285.195465,4.012120
3,43.0,2,104,49,23.02776,72.600270,110,7017,7364,1,...,0,22.694884,71.590923,120336.0,3286.0,100974.0,16076.0,143,188.400627,2.730687
5,29.0,0,123,84,-14.09402,-75.702840,139,7017,8120,2,...,0,-14.235097,-75.574821,25278.0,1546.0,0.0,23732.0,164,2592.083675,6.115990
6,43.0,2,74,49,28.61474,77.209100,4,7017,5656,1,...,0,28.646519,77.108980,238828.0,4907.0,201671.0,32250.0,97,1276.409575,2.054617
7,43.0,2,160,49,18.94017,72.834830,152,7017,666,1,...,0,19.449759,76.108221,1167496.0,31791.0,834432.0,301273.0,214,948.072083,2.723007
8,47.0,0,170,40,53.79233,13.801800,93,7017,8120,2,...,0,53.612700,12.429600,1105.0,20.0,1014.0,71.0,229,68.647398,1.809955
9,43.0,2,312,49,23.79776,87.986406,52,289,7825,1,...,0,23.814082,87.979803,218772.0,4242.0,190021.0,24509.0,410,219.630088,1.939005
10,47.0,1,31,40,49.24681,11.090850,17,7017,8120,2,...,0,48.790400,11.497900,64465.0,2648.0,56398.0,5419.0,40,492.975265,4.107655
12,51.0,0,149,84,-11.93298,-77.040850,105,7017,8120,2,...,0,-11.766533,-76.604498,349167.0,14009.0,0.0,335158.0,201,3285.195465,4.012120


Now we have to do the feature selection, I judge the selection by 3 methods: correlation coefficient, PCA, and feature importance for the boostingtree(get it after training and redo the training when drop the features)

Analaysis the correlation coefficient between features and outcome using method Spearman

In [4]:
corrdata = newdata.corr(method='spearman')['outcome']

In [5]:
corrdata.sort_values

<bound method Series.sort_values of age                       0.063901
sex                      -0.151700
province                 -0.057162
country                   0.041883
latitude                 -0.016817
longitude                -0.245517
date_confirmation         0.024868
additional_information    0.072715
source                    0.130861
outcome                   1.000000
Province_State           -0.057162
Country_Region            0.041883
Last_Update                    NaN
Lat                      -0.020569
Long_                    -0.257147
Confirmed                -0.184023
Deaths                   -0.111009
Recovered                -0.269020
Active                   -0.080459
Combined_Key             -0.063834
Incidence_Rate            0.048303
Case-Fatality_Ratio       0.232926
Name: outcome, dtype: float64>

In [6]:
newdata.head

<bound method NDFrame.head of          age  sex  province  country  latitude  longitude  date_confirmation  \
0       43.0    2       104       49  23.02776   72.60027                 74   
2       94.0    0       149       84 -12.04318  -77.02824                 74   
3       43.0    2       104       49  23.02776   72.60027                110   
5       29.0    0       123       84 -14.09402  -75.70284                139   
6       43.0    2        74       49  28.61474   77.20910                  4   
...      ...  ...       ...      ...       ...        ...                ...   
557359  56.0    1        46       84 -12.04175  -77.09082                 74   
557360  43.0    2       160       49  18.94017   72.83483                145   
557361  43.0    2       160       49  19.03681   73.01582                 95   
557362  25.0    0       279       49  13.08362   80.28252                152   
557363  43.0    2       237       49  26.26691   73.03052                 38   

        a

In [7]:
from sklearn.decomposition import PCA
pca = PCA(copy=True, n_components=8)
pca.fit(newdata.drop(columns=['outcome','age','sex']))
print(sorted(zip(map(lambda x: round(x, 6), pca.explained_variance_ratio_), newdata.drop(columns=['outcome','age','sex']).columns), reverse=True))

[(0.953733, 'province'), (0.046206, 'country'), (4.4e-05, 'latitude'), (1.4e-05, 'longitude'), (2e-06, 'date_confirmation'), (1e-06, 'additional_information'), (0.0, 'source'), (0.0, 'Province_State')]


In [8]:
corrdata[abs(corrdata) < 0.05]

country              0.041883
latitude            -0.016817
date_confirmation    0.024868
Country_Region       0.041883
Lat                 -0.020569
Incidence_Rate       0.048303
Name: outcome, dtype: float64

In [9]:
newdata = newdata.drop(columns=['Country_Region','Confirmed','province','Last_Update','country','Recovered','Deaths'])

Now split the newdata into trainset and test set with raito 80%/20%

In [10]:
trainsetRatio = 0.8

trainset = newdata.sample(frac=trainsetRatio,random_state=11,axis=0)
testset = newdata[~newdata.index.isin(trainset.index)]

In [11]:
trainset.shape

(365407, 15)

In [12]:
testset.shape

(91352, 15)

In [13]:
trainset.head(10)

,age,sex,latitude,longitude,date_confirmation,additional_information,source,outcome,Province_State,Lat,Long_,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
469998,24.0,1,48.13641,11.57754,133,7017,8120,2,31,48.790400,11.497900,5419.0,40,492.975265,4.107655
178309,43.0,2,28.61474,77.20910,85,7017,5855,3,74,28.646519,77.108980,32250.0,97,1276.409575,2.054617
366522,43.0,2,23.02776,72.60027,18,7017,6240,1,104,22.694884,71.590923,16076.0,143,188.400627,2.730687
278164,43.0,2,18.94017,72.83483,85,7017,8120,1,160,19.449759,76.108221,301273.0,214,948.072083,2.723007
336234,26.0,1,13.08362,80.28252,13,7017,4743,1,279,11.006091,78.400624,46506.0,369,682.039258,1.635877
131185,43.0,2,19.03681,73.01582,80,7017,4905,3,160,19.449759,76.108221,301273.0,214,948.072083,2.723007
532353,43.0,2,31.63347,74.87507,80,7017,6794,3,228,30.841465,75.408790,21662.0,302,307.991942,2.917066
112933,43.0,2,23.55173,74.44429,64,7017,4896,1,237,26.583423,73.847973,17717.0,312,137.339636,1.175308
386642,43.0,2,26.90000,75.80000,52,7017,5197,1,237,26.583423,73.847973,17717.0,312,137.339636,1.175308
503774,43.0,2,28.61474,77.20910,115,7017,5583,3,74,28.646519,77.108980,32250.0,97,1276.409575,2.054617


In [14]:
trainy = trainset['outcome']
trainX = trainset.drop(columns=['outcome'])
testy = testset['outcome']
testX = testset.drop(columns=['outcome'])

In [15]:
trainX

,age,sex,latitude,longitude,date_confirmation,additional_information,source,Province_State,Lat,Long_,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
469998,24.0,1,48.136410,11.577540,133,7017,8120,31,48.790400,11.497900,5419.0,40,492.975265,4.107655
178309,43.0,2,28.614740,77.209100,85,7017,5855,74,28.646519,77.108980,32250.0,97,1276.409575,2.054617
366522,43.0,2,23.027760,72.600270,18,7017,6240,104,22.694884,71.590923,16076.0,143,188.400627,2.730687
278164,43.0,2,18.940170,72.834830,85,7017,8120,160,19.449759,76.108221,301273.0,214,948.072083,2.723007
336234,26.0,1,13.083620,80.282520,13,7017,4743,279,11.006091,78.400624,46506.0,369,682.039258,1.635877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,43.0,2,18.504220,73.853020,69,7017,4897,160,19.449759,76.108221,301273.0,214,948.072083,2.723007
395094,50.0,1,28.998600,77.704420,138,7017,27,296,26.925425,80.560982,67825.0,393,144.099577,1.420411
174176,47.0,1,49.452993,8.110168,134,7017,8120,238,50.118300,7.309000,771.0,313,246.643446,2.451613
29745,43.0,2,19.870100,75.346020,125,7017,8120,160,19.449759,76.108221,301273.0,214,948.072083,2.723007


In [16]:
trainy

469998    2
178309    3
366522    1
278164    1
336234    1
         ..
883       3
395094    1
174176    2
29745     3
162370    2
Name: outcome, Length: 365407, dtype: int32

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
gbm0 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=3, random_state=0)

In [18]:
trainX.iloc[1]

age                          43.000000
sex                           2.000000
latitude                     28.614740
longitude                    77.209100
date_confirmation            85.000000
additional_information     7017.000000
source                     5855.000000
Province_State               74.000000
Lat                          28.646519
Long_                        77.108980
Active                    32250.000000
Combined_Key                 97.000000
Incidence_Rate             1276.409575
Case-Fatality_Ratio           2.054617
Name: 178309, dtype: float64

In [19]:
gbm0.fit(trainX,trainy)
y_pred = gbm0.predict(testX)
y_predprob = gbm0.predict_proba(testX)[:,1]

In [20]:
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 5), gbm0.feature_importances_), trainX.columns), reverse=True))

Features sorted by their score:
[(0.71964, 'Long_'), (0.06808, 'source'), (0.06572, 'sex'), (0.04223, 'longitude'), (0.02142, 'additional_information'), (0.01963, 'date_confirmation'), (0.01483, 'Lat'), (0.01381, 'latitude'), (0.01048, 'Incidence_Rate'), (0.01044, 'Active'), (0.00896, 'Case-Fatality_Ratio'), (0.00206, 'Province_State'), (0.00173, 'Combined_Key'), (0.00098, 'age')]


In [21]:
from sklearn.feature_selection import SelectFromModel
model = SelectFromModel(gbm0, prefit=True)
model.transform(trainX)

array([[11.4979  ],
       [77.10898 ],
       [71.590923],
       ...,
       [ 7.309   ],
       [76.108221],
       [11.4979  ]])

In [22]:
testy[0]

1

accuracy aka true ratio

In [23]:
metrics.accuracy_score(testy, y_pred)

0.8662645590682196

calculated from the precision and recall

In [24]:
metrics.f1_score(testy, y_pred,average='macro')

0.6665540130381786

ratio of predicted true values are excactly true

In [25]:
metrics.precision_score(testy, y_pred,average='macro')

0.790277137520393

ratio of true values are predicted as true

In [26]:
metrics.recall_score(testy,y_pred,average='macro')

0.644280563708466

matrix for confusion(real value VS predicted value

In [27]:
metrics.confusion_matrix(testy,y_pred)

array([[   96,   790,     7,   165],
       [   18, 36371,     1,  2969],
       [   42,     6, 32044,    17],
       [    7,  8147,    48, 10624]], dtype=int64)

Now check if the model overfitting   
I evaluate the the accuracy, precision score and f1 acore of traning dataset   
They are very close to the performance in testing dataset   
So there is no overfitting in my model  

In [28]:
y_train_pre = gbm0.predict(trainX)

In [29]:
metrics.accuracy_score(trainy, y_train_pre)

0.8674190696948881

In [30]:
metrics.f1_score(trainy, y_train_pre,average='macro')

0.6691715461992762

In [31]:
metrics.precision_score(trainy, y_train_pre,average='macro')

0.7863175538976039